1) Loading the MNIST dataset

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.datasets import fetch_openml
mnist= fetch_openml('mnist_784', version=1)
mnist.target = mnist.target.astype(np.int64)

2) Splitting the dataset into a training set and a test set 

In [2]:
X=mnist['data']
y=mnist['target']
#Considering the first 60,000 instances for training, and the rest 10,000 for testing.
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.14285) 

3) Training a Random Forest classifier on the dataset 

3) Comparing the how long it takes for training with all the features and with the principal components

In [3]:
x_train.shape

(60000, 784)

In [4]:
import timeit
start = timeit.default_timer()
from sklearn.ensemble import RandomForestClassifier
model= RandomForestClassifier()
model.fit(x_train,y_train)
stop = timeit.default_timer()
execution_time = stop-start
print('execution_time = ',execution_time)

C:\Users\banak\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


execution_time =  4.019034299999998


4) Using PCA to reduce the dataset’s dimensionality, with an explained variance ratio of 95%.

In [5]:
from sklearn.decomposition import PCA
pca=PCA(n_components=2)
#Train
principalComponents = pca.fit_transform(x_train)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])

# Test
principalComponents = pca.fit_transform(x_test)
test_principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])

5) Training a new Random Forest classifier on the reduced dataset.

In [6]:
start2 = timeit.default_timer()
from sklearn.ensemble import RandomForestClassifier
model2= RandomForestClassifier()
model2.fit(principalDf,y_train)
stop2 = timeit.default_timer()
execution_time2 = stop2-start2
print('execution_time = ',execution_time2)

C:\Users\banak\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


execution_time =  1.0282277999999998


The training is much faster with the principal components. 

6) evaluating the two classifiers on the test set

In [7]:
y_pred = model.predict(x_test)

In [8]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,model.predict(x_test))

array([[ 998,    0,    2,    1,    1,    1,    4,    0,    4,    1],
       [   0, 1098,    4,    1,    3,    0,    1,    0,    1,    0],
       [   9,    3,  951,    9,    7,    0,    6,    5,   12,    2],
       [   5,    5,   20,  931,    1,   17,    2,   11,   14,    8],
       [   2,    3,    4,    0,  885,    1,    5,    1,    3,   23],
       [  11,    3,    2,   25,    3,  875,   16,    0,   13,    5],
       [   7,    3,    4,    1,    5,    9,  969,    0,    1,    0],
       [   3,    3,   22,    1,    7,    3,    0,  985,    2,   11],
       [   6,    7,   15,   34,    8,   19,    8,    1,  845,    8],
       [   4,    3,    6,    9,   19,    7,    3,   13,   10,  921]],
      dtype=int64)

In [9]:
pd.crosstab(y_test,y_pred)

col_0,0,1,2,3,4,5,6,7,8,9
row_0,,,,,,,,,,
0,998,0,2,1,1,1,4,0,4,1
1,0,1098,4,1,3,0,1,0,1,0
2,9,3,951,9,7,0,6,5,12,2
3,5,5,20,931,1,17,2,11,14,8
4,2,3,4,0,885,1,5,1,3,23
5,11,3,2,25,3,875,16,0,13,5
6,7,3,4,1,5,9,969,0,1,0
7,3,3,22,1,7,3,0,985,2,11
8,6,7,15,34,8,19,8,1,845,8


In [10]:
y_pred2 = model2.predict(test_principalDf)

In [12]:
pd.crosstab(y_test,y_pred2)

col_0,0,1,2,3,4,5,6,7,8,9
row_0,,,,,,,,,,
0,591,0,54,14,30,43,230,2,38,10
1,2,10,4,5,424,3,4,540,1,115
2,75,8,68,40,157,112,221,136,93,94
3,21,7,44,20,353,70,107,148,55,189
4,4,48,226,418,2,52,41,11,121,4
5,77,5,98,69,139,104,240,46,110,65
6,137,6,140,104,66,146,198,35,120,47
7,4,143,349,295,12,50,36,26,111,11
8,54,3,96,57,185,115,170,99,85,87
